In [61]:
import numpy as np 
import sys

sys.path.append(r"C:\Users\abrau\uvic\seng474\assignment2\fashion-mnist-master\fashion-mnist-master\utils")
import mnist_reader

In [62]:
import gzip
import os
import numpy as np

# Define the data folder path
data_path = r"C:\Users\abrau\uvic\seng474\assignment2\fashion-mnist-master\fashion-mnist-master\data\fashion"

def load_mnist_images(filename):
    file_path = os.path.join(data_path, filename)  # Construct full path
    with gzip.open(file_path, 'rb') as f:
        f.read(16)  # Skip the header (first 16 bytes for images)
        return np.frombuffer(f.read(), dtype=np.uint8).reshape(-1, 28*28)  # Reshape to 28x28 images

def load_mnist_labels(filename):
    file_path = os.path.join(data_path, filename)  # Construct full path
    with gzip.open(file_path, 'rb') as f:
        f.read(8)  # Skip the header (first 8 bytes for labels)
        return np.frombuffer(f.read(), dtype=np.uint8)

# Load the dataset
train_images = load_mnist_images('train-images-idx3-ubyte.gz')
train_labels = load_mnist_labels('train-labels-idx1-ubyte.gz')
test_images = load_mnist_images('t10k-images-idx3-ubyte.gz')
test_labels = load_mnist_labels('t10k-labels-idx1-ubyte.gz')


# Filter training data
train_mask = (train_labels == 5) | (train_labels == 7)
train_images_filtered = train_images[train_mask]
train_labels_filtered = train_labels[train_mask]

# Filter test data
test_mask = (test_labels == 5) | (test_labels == 7)
test_images_filtered = test_images[test_mask]
test_labels_filtered = test_labels[test_mask]

In [63]:
train_labels_binary = np.where(train_labels_filtered == 5, 0, 1)
test_labels_binary = np.where(test_labels_filtered == 5, 0, 1)

train_images_normalized = train_images_filtered / 255.0
test_images_normalized = test_images_filtered / 255.0

In [64]:
# Create a subset with 1000 examples of class 0 and 1000 examples of class 1
class_0_indices = np.where(train_labels_binary == 0)[0]
class_1_indices = np.where(train_labels_binary == 1)[0]

# Randomly select 1000 examples from each class
np.random.seed(15)  # For reproducibility
subset_class_0_indices = np.random.choice(class_0_indices, 1000, replace=False)
subset_class_1_indices = np.random.choice(class_1_indices, 1000, replace=False)

# Combine the indices
subset_indices = np.concatenate([subset_class_0_indices, subset_class_1_indices])

# Shuffle the combined indices
np.random.shuffle(subset_indices)

# Create the subset
train_images_subset = train_images_normalized[subset_indices]
train_labels_subset = train_labels_binary[subset_indices]

In [65]:
import random

def random_labels(labels, noise_prob):
    noisy_labels = labels.copy()
    for i in noisy_labels:
        if random.random() < noise_prob:
            noisy_labels[i] = 1 - noisy_labels[i]

    return noisy_labels

noise_prob = 0.2
train_labels_noisy_subset = random_labels(train_labels_subset, noise_prob)
train_labels_noisy = random_labels(train_labels_binary, noise_prob)

In [66]:
train_images_flat_subset = train_images_normalized.reshape(train_images_subset.shape[0], -1)
train_images_flat = train_images_normalized.reshape(train_images_normalized.shape[0], -1)
test_images_flat = test_images_normalized.reshape(test_images_normalized.shape[0], -1)

print("Training data shape:", train_images_flat.shape)
print("Training labels shape:", train_labels_noisy.shape)
print("Training data shape:", train_images_flat_subset.shape)
print("Training labels shape:", train_labels_noisy_subset.shape)
print("Test data shape:", test_images_flat.shape)
print("Test labels shape:", test_labels_binary.shape)

Training data shape: (12000, 784)
Training labels shape: (12000,)
Training data shape: (2000, 4704)
Training labels shape: (2000,)
Test data shape: (2000, 784)
Test labels shape: (2000,)


In [67]:
np.save('train_images_flat.npy', train_images_flat)
np.save('train_labels_noisy.npy', train_labels_noisy)
np.save('train_images_flat_subset.npy', train_images_flat_subset)
np.save('train_labels_noisy_subset.npy', train_labels_noisy_subset)
np.save('test_images_flat.npy', test_images_flat)
np.save('test_labels_binary.npy', test_labels_binary)